In [1]:
#Dependencies
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import requests
from config import *
from bs4 import BeautifulSoup
print(api_key)

xyc


In [2]:
#import csv
og_df = pd.read_csv("Resources/results.csv")
og_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [3]:
#names of all columns
print(og_df.columns)

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId'],
      dtype='object')


In [4]:
#new dataframe without other columns from csv

fatal_df = og_df.drop(columns = ['grid', 'fastestLapSpeed', 'fastestLapTime', 'fastestLap', 'milliseconds', 'time', 'laps', 'points', 'positionOrder', 'positionText'])
fatal_df.head()

,resultId,raceId,driverId,constructorId,number,position,rank,statusId
0,1,18,1,1,22,1,2,1
1,2,18,2,2,3,2,3,1
2,3,18,3,3,7,3,5,1
3,4,18,4,4,5,4,7,1
4,5,18,5,1,23,5,1,1


In [17]:
fatal_df.describe()

,resultId,raceId,driverId,constructorId,statusId
count,25840.000000,25840.000000,25840.000000,25840.000000,25840.000000
mean,12921.334327,531.425813,261.732082,48.628328,17.568576
std,7460.682031,299.440908,268.623016,59.732131,26.155520
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,6460.750000,293.000000,56.000000,6.000000,1.000000
50%,12920.500000,514.000000,163.000000,25.000000,11.000000
75%,19380.250000,784.000000,360.000000,58.000000,14.000000
max,25845.000000,1096.000000,856.000000,214.000000,141.000000


In [5]:
#np.arange(8) == np.arange(8).astype(str)
fatal_df.dtypes

resultId          int64
raceId            int64
driverId          int64
constructorId     int64
number           object
position         object
rank             object
statusId          int64
dtype: object

In [52]:
fatal_2_df = 0
for i in fatal_df["statusId"]:
    if i == 104:
        fatal_2_df = fatal_2_df+1

print(fatal_2_df)

NameError: name 'fatal_df' is not defined

In [64]:
#API for wikipedia data
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_fatalities"

page = requests.get(url)
soup  = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='wikitable plainrowheaders sortable')

fatalitiesdf = pd.read_html(str(table))
fatalitiesdf = pd.concat(fatalitiesdf)
df_new = fatalitiesdf.drop(fatalitiesdf[(fatalitiesdf['Session'] == 'Test') | (fatalitiesdf['Session'] == 'Practice') | (fatalitiesdf['Session'] == 'Qualifying') | (fatalitiesdf['Session'] == 'Run')].index)
df_new.index = pd.to_datetime(df_new['Date of accident'], format='%B %d, %Y')

grouped_set = df_new.groupby(by=[df_new.index.year])['Driver'].count()

Date of accident
1955-04-11             Mario Alborghetti (ITA)
1955-05-30                 Bill Vukovich (USA)
1958-05-30                  Pat O'Connor (USA)
1958-07-06                   Luigi Musso (ITA)
1958-08-03                  Peter Collins (UK)
1958-10-19     Stuart Lewis-Evans (UK)[note 3]
1960-06-19                  Chris Bristow (UK)
1960-06-19                    Alan Stacey (UK)
1961-09-10    Wolfgang von Trips (FRG)[note 7]
1966-08-07            John Taylor (UK)[note 9]
1967-05-07      Lorenzo Bandini (ITA)[note 10]
1968-07-07                  Jo Schlesser (FRA)
1970-05-25                   Martin Brain (UK)
1970-06-21                  Piers Courage (UK)
1971-10-24                    Jo Siffert (SUI)
1973-07-29               Roger Williamson (UK)
1974-10-06               Helmuth Koinigg (AUT)
1977-03-05             Tom Pryce (UK)[note 12]
1978-09-10      Ronnie Peterson (SWE)[note 14]
1982-06-13              Riccardo Paletti (ITA)
1994-05-01                  Ayrton Senna (B